#### Import required libraries
##### Set path interface. Please change to your own path

In [1]:
import sys
#sys.path.append('/home/olexandr/notebooks/ASE_ANI/lib')
from ase_interface import ANIENS
from ase_interface import aniensloader

van Der Waals correction will be unavailable. Please install ased3


In [2]:
import numpy as np
import time
# ASE
import  ase
from ase.io import read, write
from ase.optimize import BFGS, LBFGS 
from ase.vibrations import Vibrations
from ase.thermochemistry import IdealGasThermo

#figure plotting
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
#import seaborn as sns
%matplotlib inline

#### Read geometry from xyz file and setup calculator

In [3]:
geometry = read('data/water.xyz')

# Current ANI model options are:
# '../ani_models/ani-1ccx_8x.info' Coupled cluster transfer learned model
# '../ani_models/ani-1x_8x.info'   Full ANI-1x wb97x/6-31g* dataset model
geometry.set_calculator(ANIENS(aniensloader('../ani_models/ani-1ccx_8x.info', gpu=0, multigpu=False)))

#### Calculate single point energy, atomic energies, and forces

In [4]:
e = geometry.get_potential_energy()
print('Total energy', e, 'eV')

Total energy -2078.50282282132 eV


In [5]:
geometry.calc.get_atomicenergies(sae=False)

array([[-0.30596783, -0.18889855, -0.18626961],
       [-0.3118565 , -0.17750212, -0.17757316],
       [-0.2737363 , -0.20476448, -0.20041333],
       [-0.38988734, -0.14600253, -0.14439384],
       [-0.49649216, -0.09650336, -0.09329748],
       [-0.39089172, -0.1363589 , -0.13453905],
       [-0.39495803, -0.14175725, -0.13636874],
       [-0.48994318, -0.09549404, -0.0930541 ]])

In [6]:
geometry.get_forces()

array([[ 3.5510799e-01,  8.3285220e-02, -2.9023369e-03],
       [-7.4988507e-02,  1.1935695e-02, -1.1308717e-04],
       [-2.8011954e-01, -9.5221400e-02,  3.0154386e-03]], dtype=float32)

#### Geometry optimization with BFGS

In [7]:
start_time = time.time()
dyn = LBFGS(geometry)
dyn.run(fmax=0.001)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

       Step     Time          Energy         fmax
LBFGS:    0 15:54:00    -2078.502823        0.3648
LBFGS:    1 15:54:00    -2078.503865        0.1675
LBFGS:    2 15:54:00    -2078.504142        0.0507
LBFGS:    3 15:54:00    -2078.504212        0.0296
LBFGS:    4 15:54:00    -2078.504264        0.0128
LBFGS:    5 15:54:00    -2078.504266        0.0017
LBFGS:    6 15:54:00    -2078.504266        0.0001
[ANI Total time: 0.04100346565246582 seconds]


In [8]:
e = geometry.get_potential_energy()
print('Total energy', e, 'eV')

Total energy -2078.5042660113986 eV


In [9]:
geometry.get_forces()

array([[-2.6337273e-05, -1.3322434e-04,  3.3407159e-06],
       [ 3.5057892e-06,  9.5589035e-05, -2.3597306e-06],
       [ 2.2613500e-05,  3.8772163e-05, -1.0102135e-06]], dtype=float32)

#### Vibrations and thermochemistry with numeric 2nd derivatives

In [10]:
vib = Vibrations(geometry, nfree=2) 
vib.run()

Writing vib.eq.pckl
Writing vib.0x-.pckl
Writing vib.0x+.pckl
Writing vib.0y-.pckl
Writing vib.0y+.pckl
Writing vib.0z-.pckl
Writing vib.0z+.pckl
Writing vib.1x-.pckl
Writing vib.1x+.pckl
Writing vib.1y-.pckl
Writing vib.1y+.pckl
Writing vib.1z-.pckl
Writing vib.1z+.pckl
Writing vib.2x-.pckl
Writing vib.2x+.pckl
Writing vib.2y-.pckl
Writing vib.2y+.pckl
Writing vib.2z-.pckl
Writing vib.2z+.pckl


In [11]:
vib.summary()

---------------------
  #    meV     cm^-1
---------------------
  0    2.0i     15.8i
  1    1.1i      9.1i
  2    0.1i      1.0i
  3    0.3       2.6 
  4    3.4      27.0 
  5    3.5      28.5 
  6  213.7    1723.3 
  7  474.9    3830.1 
  8  477.9    3854.7 
---------------------
Zero-point energy: 0.587 eV


In [12]:
vib.get_zero_point_energy()

0.5868330720915512

In [13]:
vib.get_frequencies()

array([0.00000000e+00+15.76858108j, 0.00000000e+00 +9.09943649j,
       0.00000000e+00 +0.99467795j, 2.63641158e+00 +0.        j,
       2.70370567e+01 +0.        j, 2.85366076e+01 +0.        j,
       1.72330433e+03 +0.        j, 3.83006694e+03 +0.        j,
       3.85467458e+03 +0.        j])

In [14]:
vib_energies = vib.get_energies()

thermo = IdealGasThermo(vib_energies=vib_energies,
                        potentialenergy=e,
                        atoms=geometry,
                        geometry='nonlinear',
                        symmetrynumber=1, spin=0)
G = thermo.get_gibbs_energy(temperature=298.15, pressure=101325.)

Enthalpy components at T = 298.15 K:
E_pot              -2078.504 eV
E_ZPE                  0.583 eV
Cv_trans (0->T)        0.039 eV
Cv_rot (0->T)          0.039 eV
Cv_vib (0->T)          0.000 eV
(C_v -> C_p)           0.026 eV
-------------------------------
H                  -2077.818 eV

Entropy components at T = 298.15 K and P = 101325.0 Pa:
                           S               T*S
S_trans (1 atm)    0.0015008 eV/K        0.447 eV
S_rot              0.0005130 eV/K        0.153 eV
S_elec             0.0000000 eV/K        0.000 eV
S_vib              0.0000002 eV/K        0.000 eV
S (1 atm -> P)    -0.0000000 eV/K       -0.000 eV
-------------------------------------------------
S                  0.0020140 eV/K        0.600 eV

Free energy components at T = 298.15 K and P = 101325.0 Pa:
    H      -2077.818 eV
 -T*S         -0.600 eV
-----------------------
    G      -2078.419 eV


In [15]:
G

-2078.41869725461

In [16]:
# remove mode pckl files
vib.clean()

19